In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tabulate import tabulate

In [6]:
ga_path = '../pickles/ga/'
kbga_path = '../pickles/kbga/corrected/'

In [7]:
ga_files = ['n_run_az_100fc_ts.pkl', 
            "n_run_az_ga_100fc_copt_avg.pkl"]

In [8]:
kbga_files = ['n_run_az_kbga_es50_100.pkl',
              'n_run_az_kbga_vp.pkl',
              'n_run_az_kbga_es10_100.pkl',
              'n_run_az_kbga_nokbps.pkl', 
              'n_run_az_kbga_cmp.pkl']

In [51]:
def average_best_fitness_value(az, imdb, yelp):
    cols = ["gen_"+str(i) for i in [1, 25, 50, 75, 100]]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]
    
    data = [
        az.iloc[-1, [0, 24, 49, 74, 99]],
        imdb.iloc[-1, [0, 24, 49, 74, 99]],
        yelp.iloc[-1, [0, 24, 49, 74, 99]],
        ]
    
    abfv = pd.DataFrame(data, columns=cols)
    abfv = abfv.append(abfv.mean(), ignore_index=True)

    abfv.index = index_col
    abfv.index.name = "Datasets"
    
    return abfv

In [84]:
def average_best_of_generation(az, imdb, yelp):
    cols = ["1-25", "26-50", "51-75", "76-100"]
    index_col = ["Amazon", "IMDB", "Yelp", "Average"]
    
    data = [
        [az.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
        [imdb.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
        [yelp.iloc[-1, i-25:i].mean() for i in range(25, 101, 25)],
    ]

    abog = pd.DataFrame(data, columns=cols)
    abog = abog.append(abog.mean(), ignore_index=True)

    abog.index = index_col
    abog.index.name = "Datasets"
    return abog

In [ ]:
def optimization_accuracy():
    pass

In [ ]:
def likelihood_of_evolution_leap():
    pass

In [ ]:
def probability_of_convergence():
    pass

In [ ]:
def average_no_of_function_evaluations():
    pass

In [ ]:
def successful_performance():
    pass

In [ ]:
def t_test():
    pass

In [11]:
def tabulate_runs(save_path, runs):
    with open(save_path, 'rb') as gf:
        data = pickle.load(gf)
        chromos = []
        scores = []
        exec_time = []
        for run in data:
            chromos.append(run[0])
            scores.append(run[1])
            exec_time.append(run[2])

    df = pd.DataFrame()
    cols = ["gen_" + str(i) for i in range(1, 101)]
    df[cols] = pd.DataFrame(scores)

    df = pd.concat([df, pd.DataFrame({'exec_time': exec_time})], axis=1)
    df = df.append(df.mean(), ignore_index=True)

    df.index.name = 'runs'
    indexes = ["run_"+str(i) for i in range(1, runs+1)] + ['Average']
    df.index = indexes
    
    return df

In [63]:
ga_az = tabulate_runs(ga_path+"corrected/"+ga_files[0], 5)
kbga_az = tabulate_runs(kbga_path+kbga_files[1], 5)

In [65]:
kbga_az

,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,gen_10,...,gen_92,gen_93,gen_94,gen_95,gen_96,gen_97,gen_98,gen_99,gen_100,exec_time
run_1,0.6640,0.6920,0.700,0.728,0.7280,0.7560,0.7600,0.7600,0.7640,0.7640,...,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.9000,0.900,23953.003853
run_2,0.6880,0.6960,0.688,0.712,0.7200,0.7200,0.7440,0.7440,0.7480,0.7520,...,0.8880,0.8880,0.8880,0.8880,0.8920,0.8920,0.8920,0.8920,0.892,22758.824567
run_3,0.6880,0.6960,0.708,0.712,0.7240,0.7360,0.7520,0.7520,0.7640,0.7680,...,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.8920,0.896,40535.096549
run_4,0.6640,0.6880,0.708,0.752,0.7520,0.7600,0.7640,0.7760,0.7880,0.7880,...,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.896,40335.615604
run_5,0.6800,0.6840,0.736,0.736,0.7640,0.7640,0.7760,0.7760,0.7840,0.7840,...,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.8960,0.896,34065.702112
Average,0.6768,0.6912,0.708,0.728,0.7376,0.7472,0.7592,0.7616,0.7696,0.7712,...,0.8944,0.8944,0.8944,0.8944,0.8952,0.8952,0.8952,0.8952,0.896,32329.648537


## Measures

In [57]:
abfv_ga = average_best_fitness_value(ga_az, ga_az, ga_az)
abfv_kbga = average_best_fitness_value(kbga_az, kbga_az, kbga_az)
abfv = pd.concat([abfv_ga, abfv_kbga], axis=1)
abfv

,gen_1,gen_25,gen_50,gen_75,gen_100,gen_1,gen_25,gen_50,gen_75,gen_100
Datasets,,,,,,,,,,
Amazon,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
IMDB,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
Yelp,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896
Average,0.6336,0.7248,0.7464,0.764,0.7696,0.6768,0.8368,0.8736,0.888,0.896


In [85]:
abog_ga = average_best_of_generation(ga_az, ga_az, ga_az)
abog_kbga = average_best_of_generation(kbga_az, kbga_az, kbga_az)
abog = pd.concat([abog_ga, abog_kbga], axis=1)
abog

,1-25,26-50,51-75,76-100,1-25,26-50,51-75,76-100
Datasets,,,,,,,,
Amazon,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
IMDB,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
Yelp,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
Average,0.696384,0.738208,0.756352,0.766944,0.781792,0.859904,0.881696,0.892896
